In [13]:
import pandas as pd
import numpy as np
import random
import pytz
import datetime
import ast
import os

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.dates import DateFormatter
import plotly.express as px
import plotly.graph_objects as go
import missingno as msno
import folium
from geopy.geocoders import Nominatim
import ipywidgets as widgets
from folium.plugins import AntPath
from IPython.display import display, clear_output, IFrame, HTML

import warnings
warnings.filterwarnings("ignore")

In [14]:
df_torch_route = pd.read_csv("D:/Abhinav/Test/ADA/2024_olympics/torch_route.csv")
df_venues = pd.read_csv("D:/Abhinav/Test/ADA/2024_olympics/venues.csv")

In [15]:
df_torch_route.head(1)

,title,city,date_start,date_end,tag,url,stage_number
0,Lighting Ceremony,Olympia,2024-04-15T22:01:00Z,2024-04-16T10:01:00Z,lighting-ceremony,https://olympics.com/en/paris-2024/olympic-tor...,NaN


In [16]:
df_venues.head(1)

,venue,sports,date_start,date_end,tag,url
0,Aquatics Centre,"['Artistic Swimming', 'Diving', 'Water Polo']",2024-07-27T09:00:00Z,2024-08-10T20:00:00Z,aquatics-centre,https://olympics.com/en/paris-2024/venues/aqua...


`geopy` library is used to geocode the cities to get their latitude and longitude.
`folium` library is used for mapping.

In [17]:
geolocator = Nominatim(user_agent='torch_route')

def geocode_city(city):
    try:
        location = geolocator.geocode(city + ", France")
        return(location.latitude, location.longitude)
    except:
        return None

In [18]:
df_torch_route['coords'] = df_torch_route['city'].apply(geocode_city)
df_torch_route.head()

,title,city,date_start,date_end,tag,url,stage_number,coords
0,Lighting Ceremony,Olympia,2024-04-15T22:01:00Z,2024-04-16T10:01:00Z,lighting-ceremony,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(48.8702443, 2.3283595)"
1,Relay in Greece,NaN,2024-04-16T10:02:00Z,2024-04-26T17:06:00Z,relay-in-greece,https://olympics.com/en/paris-2024/olympic-tor...,NaN,None
2,Handover Ceremony,Athens,2024-04-25T22:01:00Z,2024-04-26T17:07:00Z,handover-ceremony,https://olympics.com/en/paris-2024/olympic-tor...,NaN,None
3,Crossing the Mediterranean,Marseille,2024-04-26T17:08:00Z,2024-05-07T22:01:00Z,mediterranean-crossing,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(43.2961743, 5.3699525)"
4,Prologue,Marseille,2024-05-07T22:02:00Z,2024-05-08T20:02:00Z,prologue-marseille,https://olympics.com/en/paris-2024/olympic-tor...,NaN,"(43.2961743, 5.3699525)"


💡 It's been noticed that GeoCoding failed (4) times, let's investigate that.

In [19]:
print("Before dropping NaN rows: ", df_torch_route.shape)
nan_rows = df_torch_route[df_torch_route['coords'].isna()]
print("Rows with NaN values in 'coords column:")
nan_rows.head()

Before dropping NaN rows:  (73, 8)
Rows with NaN values in 'coords column:


,title,city,date_start,date_end,tag,url,stage_number,coords
1,Relay in Greece,NaN,2024-04-16T10:02:00Z,2024-04-26T17:06:00Z,relay-in-greece,https://olympics.com/en/paris-2024/olympic-tor...,NaN,None
2,Handover Ceremony,Athens,2024-04-25T22:01:00Z,2024-04-26T17:07:00Z,handover-ceremony,https://olympics.com/en/paris-2024/olympic-tor...,NaN,None
20,Vienne,Grand Poitiers Futuroscope,2024-05-24T22:01:00Z,2024-05-25T21:59:00Z,vienne-grand-poitiers-futuroscope,https://olympics.com/en/paris-2024/olympic-tor...,16.0,None
35,French Polynesia,Pīra'e-Pape'ete,2024-06-12T22:01:00Z,2024-06-13T21:59:00Z,polynesie-francaise-papeete,https://olympics.com/en/paris-2024/olympic-tor...,31.0,None


💡 It has been noticed that the geocoding failed:

- For `Relay in Greece` and `Handover Ceremony in Athena` the geocoding failed because these locations are not in Paris. The flame burned at the ancient Acropolis in Athens, a week before its handover to the Paris 2024 organizers. The ritual began at Ancient Olympia, the original site of the Olympic Games, with the lighting of the Olympic flame. This tradition continues today, marking the start of each Olympic Games' preparations. The lighting ceremony took place several months before the Olympics, near the temple of Hera in Olympia.

- For `Pyrénées-Atlantiques`, `Vienne` and `French Polynesia`, we either manually fill in the location details, as they were not recognized by the Geopy library, or drop them

In [20]:
# # Ensure coords column is of type string
#df_torch_route['coords'] = df_torch_route['coords'].astype(str)

# # Update coordinates for specific titles
#df_torch_route.loc[df_torch_route['title'] == 'Vienne', 'coords'] = '(45.528, 4.874)'
#df_torch_route.loc[df_torch_route['title'] == 'French Polynesia', 'coords'] = '(17.6797, -149.4068)'

In [21]:
df_torch_route = df_torch_route.dropna(subset=['coords'])
print("After dropping NaN rows:", df_torch_route.shape)

After dropping NaN rows: (69, 8)


`Draw the Route🔮`

This code creates an interactive map that visualizes the Olympic torch relay route, including markers, arrows, and custom prompots for each city.

- **Distinct Markers for Key Cities**: The first and last cities on the map are highlighted with star markers to differentiate them from other cities, providing clear visual cues for these significant points.

- **Custom Pop-Up Messages**: Clicking on each city triggers a customized pop-up message, displaying detailed information about the city and the Olympic Torch route.

- **Interactive Navigation**: The map supports interactive navigation, allowing to move and zoom using keyboard (+,-,arrows) controls, offering a more intuitive and accessible experience for exploring the map.

In [26]:
df_torch_route['date_start'] = pd.to_datetime(df_torch_route['date_start'], errors='coerce')
df_torch_route['date_end'] = pd.to_datetime(df_torch_route['date_end'], errors='coerce')

m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

colors = ['red', 'blue', 'green', 'orange', 'purple', 'darkred', 'lightblue', 'darkgreen', 'lightgreen', 'beige']

df_torch_route = df_torch_route.sort_values('date_start').reset_index(drop=True)

for i, row in df_torch_route.iterrows():
    popup_message = (
        f"<div style='width: 300px; font-size: 14px;'>"
        f"<b>Stage:</b> {i + 1}<br>"  # Stage is 1-based index
        f"<b>Title:</b> {row['title']}<br>"
        f"<b>City:</b> {row['city']}<br>"
        f"<b>Start Date:</b> {row['date_start'].strftime('%Y-%m-%d %H:%M:%S')}<br>"
        f"<b>End Date:</b> {row['date_end'].strftime('%Y-%m-%d %H:%M:%S')}<br>"
        f"<b>Tag:</b> {row['tag']}<br>"
        # f"<b>URL:</b> {row['url']}"
        f"</div>"
    )

    color = colors[i % len(colors)]

    if i == 0:
        icon = folium.Icon(color='black', icon='star', prefix='fa')  
    elif i == len(df_torch_route) - 1:
        icon = folium.Icon(color='black', icon='star', prefix='fa') 
    else:
        icon = folium.Icon(color=color)  

    folium.Marker(
        location=row['coords'],
        icon=icon,
        popup=folium.Popup(popup_message, max_width=300)  
    ).add_to(m)

for i in range(len(df_torch_route) - 1):
    start = df_torch_route.iloc[i]['coords']
    end = df_torch_route.iloc[i + 1]['coords']
    color = colors[i % len(colors)] 

    AntPath(
        locations=[start, end],
        dash_array=[20, 20],
        delay=1000,
        color=color,
        pulse_color=color,
        reverse=True
    ).add_to(m)

# Add a legend
legend_html = '''
    <div style="
        position: fixed;
        bottom: 50px; left: 50px; width: 300px; height: auto;
        background-color: white; border:2px solid black;
        z-index:9999; font-size:14px;
        ">
        <b>Legend</b><br>
        <i style="background:red"></i> City 1<br>
        <i style="background:blue"></i> City 2<br>
        <i style="background:green"></i> City 3<br>
        <!-- Add more legend entries here -->
    </div>
    '''
m.get_root().add_child(folium.Element(legend_html))

custom_css = """
<style>
#map {
    width: 100%;
    height: 100vh;
}
</style>
"""
m.get_root().html.add_child(folium.Element(custom_css))

custom_js = """
<script>
document.addEventListener('keydown', function(event) {
    var map = document.querySelector('#map')._leaflet_map;
    if (event.key === '+') {
        map.zoomIn();
    } else if (event.key === '-') {
        map.zoomOut();
    }
});
</script>
"""
m.get_root().html.add_child(folium.Element(custom_js))

# m.save('torch_route_map.html')
display(m)

Upon review, it has come to our attention that four cities listed in the Torch relay are located outside mainland France. These cities are:

- `Baie-Mahault`: This is a prominent town situated in Guadeloupe, which is an overseas region of France.
- `Fort-de-France`: This is both a commune and the capital city of Martinique, an overseas department and region of France located in the Caribbean.
- `Cayenne`: This city is the prefecture of French Guiana, an overseas region and department of France located in South America.
- `Nouméa`: This city is the capital of New Caledonia, an overseas country of France located in the southwestern Pacific Ocean.


`Interactive Olympic Sport-Venue Selector`

Paris gears up for the Olympic Games Paris 2024. The French capital hosts the world’s biggest sports festival. This edition is bigger than ever, with as many as `32 sports` disciplines. All these sports, of course, need a stadium or venue. In the city itself, near Paris, and further across the country, there are some `35 stadiums or venues` where a sport is held.


In [27]:
df_venues.head()

,venue,sports,date_start,date_end,tag,url
0,Aquatics Centre,"['Artistic Swimming', 'Diving', 'Water Polo']",2024-07-27T09:00:00Z,2024-08-10T20:00:00Z,aquatics-centre,https://olympics.com/en/paris-2024/venues/aqua...
1,Bercy Arena,"['Artistic Gymnastics', 'Basketball', 'Trampol...",2024-07-27T09:00:00Z,2024-08-11T16:00:00Z,bercy-arena,https://olympics.com/en/paris-2024/venues/berc...
2,Bordeaux Stadium,['Football'],2024-07-25T17:00:00Z,2024-08-02T21:59:00Z,bordeaux-stadium,https://olympics.com/en/paris-2024/venues/bord...
3,Champ de Mars Arena,"['Judo', 'Wrestling']",2024-07-27T08:00:00Z,2024-08-11T12:00:00Z,champ-de-mars-arena,https://olympics.com/en/paris-2024/venues/cham...
4,Château de Versailles,"['Equestrian', 'Modern Pentathlon']",2024-07-27T07:30:00Z,2024-08-11T11:30:00Z,chateau-de-versailles,https://olympics.com/en/paris-2024/venues/chat...


In [28]:
def safe_literal_eval(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return value
    
df_venues['sports'] = df_venues['sports'].apply(safe_literal_eval)
df_venues = df_venues.explode('sports')
df_venues.head()

,venue,sports,date_start,date_end,tag,url
0,Aquatics Centre,Artistic Swimming,2024-07-27T09:00:00Z,2024-08-10T20:00:00Z,aquatics-centre,https://olympics.com/en/paris-2024/venues/aqua...
0,Aquatics Centre,Diving,2024-07-27T09:00:00Z,2024-08-10T20:00:00Z,aquatics-centre,https://olympics.com/en/paris-2024/venues/aqua...
0,Aquatics Centre,Water Polo,2024-07-27T09:00:00Z,2024-08-10T20:00:00Z,aquatics-centre,https://olympics.com/en/paris-2024/venues/aqua...
1,Bercy Arena,Artistic Gymnastics,2024-07-27T09:00:00Z,2024-08-11T16:00:00Z,bercy-arena,https://olympics.com/en/paris-2024/venues/berc...
1,Bercy Arena,Basketball,2024-07-27T09:00:00Z,2024-08-11T16:00:00Z,bercy-arena,https://olympics.com/en/paris-2024/venues/berc...


This code allows to filter by `sport` using a dropdown menu. Upon selection, it displays the specific URL for a page designed by the Olympics committee for detailed information about the sport and the French venue dedicated to hosting it.

In [30]:
dropdown = widgets.Dropdown(
    options=['Please choose a sport'] + list(df_venues['sports'].unique()),
    value='Please choose a sport',
    description='Sport:',
    disabled=False
)

output = widgets.Output()

def on_change(change):
    with output:
        clear_output(wait=True)
        sport = change.new
        if sport == 'Please choose a sport':
            return

        selected_row = df_venues[df_venues['sports'] == sport]
        if not selected_row.empty:
            venue_url = selected_row['url'].values[0]

            display(IFrame(src=venue_url, width='100%', height='700px'))

dropdown.observe(on_change, names='value')

#display(dropdown, output)
